In [1]:
import pyspark
import time
from pyspark.sql import SparkSession
from pyspark.sql.functions import col
from pyspark.ml.regression import LinearRegression
from pyspark.ml.feature import VectorAssembler
import matplotlib.pyplot as plt
import numpy as np
import h5py
import pandas as pd
import glob
from pyspark.sql.types import StructType, StructField, StringType, FloatType, IntegerType

In [1]:
from pyspark.sql import SparkSession

spark_session = SparkSession\
        .builder\
        .master("spark://192.168.2.130:7077") \
        .appName("de16_sparky")\
        .config("spark.dynamicAllocation.enabled", True)\
        .config("spark.dynamicAllocation.shuffleTracking.enabled",True)\
        .config("spark.shuffle.service.enabled", False)\
        .config("spark.dynamicAllocation.executorIdleTimeout","300s")\
        .config("spark.executor.cores",2)\
        .config("spark.driver.port",9999)\
        .config("spark.blockManager.port",10005)\
        .getOrCreate()


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/03/07 10:16:50 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/03/07 10:16:51 WARN Utils: Service 'sparkDriver' could not bind on port 9999. Attempting port 10000.
24/03/07 10:16:51 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
24/03/07 10:16:52 WARN Utils: Service 'org.apache.spark.network.netty.NettyBlockTransferService' could not bind on port 10005. Attempting port 10006.


In [3]:
import org.apache.hadoop.fs.FileSystem;
import org.apache.hadoop.fs.Path;

# Get Spark's Hadoop configuration
conf = spark_session.sparkContext.hadoopConfiguration

# Create a FileSystem instance
fs = FileSystem.get(conf)

# Check if file exists
file_path = "hdfs://192.168.2.130:9000/user/MillionSongSubset/A/A/A/TRAAAAW128F429D538.h5"
exists = fs.exists(Path(file_path))

if exists:
    print("File exists in HDFS.")
else:
    print("File does not exist in HDFS.")


ModuleNotFoundError: No module named 'org'

In [3]:
start_time = time.time()

In [25]:
def read_h5_to_row(filename):
    print("inside func")
    """
    Read a single HDF5 file and return a tuple representing a row of data.
    """
    print(filename)
    try:
        with h5py.File(filename, 'r') as file:
            return (

                float(file['analysis']['songs'][0][23]),

                int(file['musicbrainz']['songs'][0][1]),
            )
    except Exception as e:
        print(f"Error processing file {filename}: {e}")
        return None

In [23]:

hdfs_path = 'hdfs://192.168.2.130:9000/user/MillionSongSubset'
song_data = spark.read.format("hdf5").load(hdfs_path)
song_paths = glob.glob('hdfs://192.168.2.130:9000/user/MillionSongSubset/*/*/*/*.h5')

# Use Spark to parallelize the file processing and read the data
song_data_rdd = spark_session.sparkContext.parallelize(song_paths[:8000]).map(read_h5_to_row)

print(f'Partitions: {song_data_rdd.getNumPartitions()}')
print(f'Partitions: {song_data_rdd.getNumPartitions()}')

# Convert RDD to Spark DataFrame
schema = StructType([

    StructField("loudness", FloatType(), True),

    StructField("year", IntegerType(), True)
])

columns = ['loudness', 'year']
df = spark_session.createDataFrame(song_data_rdd, schema=schema)
print(f'Partitions: {df.rdd.getNumPartitions()}')

NameError: name 'spark' is not defined

In [34]:
hdfs_path = "hdfs://192.168.2.130:9000/user/MillionSongSubset/A/A/A/TRAAAAW128F429D538.h5"
file = spark_session.sparkContext.textFile(hdfs_path)
read_h5_to_row(hdfs_path)


inside func
hdfs://192.168.2.130:9000/user/MillionSongSubset/A/A/A/TRAAAAW128F429D538.h5
Error processing file hdfs://192.168.2.130:9000/user/MillionSongSubset/A/A/A/TRAAAAW128F429D538.h5: [Errno 2] Unable to synchronously open file (unable to open file: name = 'hdfs://192.168.2.130:9000/user/MillionSongSubset/A/A/A/TRAAAAW128F429D538.h5', errno = 2, error message = 'No such file or directory', flags = 0, o_flags = 0)


In [31]:
hdfs_path = 'hdfs://192.168.2.130:9000/user/MillionSongSubset'
file_paths_rdd = spark_session.sparkContext.wholeTextFiles('hdfs://192.168.2.130:9000/user/MillionSongSubset/*/*/*/*.h5')
file_paths = file_paths_rdd.keys().map(read_h5_to_row)

In [5]:
for item in song_data_rdd.take(5):
    print(item)

In [6]:
# df = spark_session.read.csv("test.csv", header=True, inferSchema=True)#.cache()
# df.show(5)
filtered_df = df.filter(col("year") != 0)
# filtered_df = filtered_df.repartition(10)
print(f'Partitions: {filtered_df.rdd.getNumPartitions()}')
# df.show()

Partitions: 2


In [7]:

assembler = VectorAssembler(inputCols=["year"], outputCol="features")
assembled_df = assembler.transform(filtered_df)
# assembled_df.show(10, truncate= False)

In [8]:
lr = LinearRegression(featuresCol="features", labelCol="loudness")
model = lr.fit(assembled_df)
coefficients = [model.coefficients[0], model.intercept]

end_time = time.time()
duration = end_time - start_time


print("Coefficients:", coefficients)
print(f'execution time: {duration}')
print(f'Partitions: {assembled_df.rdd.getNumPartitions()}')
print(f'number of cores: {cores}')

24/03/07 09:20:10 WARN Instrumentation: [4b677442] regParam is zero, which might cause numerical instability and overfitting.
24/03/07 09:20:13 ERROR Instrumentation: java.lang.AssertionError: assertion failed: Training dataset is empty.
	at scala.Predef$.assert(Predef.scala:223)
	at org.apache.spark.ml.optim.WeightedLeastSquares$Aggregator.validate(WeightedLeastSquares.scala:425)
	at org.apache.spark.ml.optim.WeightedLeastSquares.fit(WeightedLeastSquares.scala:108)
	at org.apache.spark.ml.regression.LinearRegression.trainWithNormal(LinearRegression.scala:456)
	at org.apache.spark.ml.regression.LinearRegression.$anonfun$train$1(LinearRegression.scala:354)
	at org.apache.spark.ml.util.Instrumentation$.$anonfun$instrumented$1(Instrumentation.scala:191)
	at scala.util.Try$.apply(Try.scala:213)
	at org.apache.spark.ml.util.Instrumentation$.instrumented(Instrumentation.scala:191)
	at org.apache.spark.ml.regression.LinearRegression.train(LinearRegression.scala:329)
	at org.apache.spark.ml.re

Py4JJavaError: An error occurred while calling o100.fit.
: java.lang.AssertionError: assertion failed: Training dataset is empty.
	at scala.Predef$.assert(Predef.scala:223)
	at org.apache.spark.ml.optim.WeightedLeastSquares$Aggregator.validate(WeightedLeastSquares.scala:425)
	at org.apache.spark.ml.optim.WeightedLeastSquares.fit(WeightedLeastSquares.scala:108)
	at org.apache.spark.ml.regression.LinearRegression.trainWithNormal(LinearRegression.scala:456)
	at org.apache.spark.ml.regression.LinearRegression.$anonfun$train$1(LinearRegression.scala:354)
	at org.apache.spark.ml.util.Instrumentation$.$anonfun$instrumented$1(Instrumentation.scala:191)
	at scala.util.Try$.apply(Try.scala:213)
	at org.apache.spark.ml.util.Instrumentation$.instrumented(Instrumentation.scala:191)
	at org.apache.spark.ml.regression.LinearRegression.train(LinearRegression.scala:329)
	at org.apache.spark.ml.regression.LinearRegression.train(LinearRegression.scala:186)
	at org.apache.spark.ml.Predictor.fit(Predictor.